# Healthy Personalized Food Recommender System Using Two-Tower Deep Neural Network

# Motivation 

Recommender systems are designed to assist users in discovering relevant items by analyzing their preferences and generating a tailored list of suggestions. These systems effectively address the problem of choice overload, enabling users to navigate through vast amount of options with ease. By accurately predicting user preferences, recommender systems enhance user experience and satisfaction. They are widely employed across various domains, offering personalized recommendations that cater to individual needs and interest.
Recommender system employs three different approaches to generate recommendations, collaborative filtering, content based filtering and hybrid approach. 


In the domain of food, recommender systems are utilized in various contexts such as restaurant suggestions, grocery store item personalization, and recipe recommendations. Creating an accurate food recommender system is complex due to the multifaceted nature of the food domain, which involves context, location, health preferences, and more.


## Problem Formulation

Deciding what and where to eat is challenging due to the vast array of available options. Online food recommender systems aim to help users find relevant food items and recipes. However, previous studies show that these systems often promote unhealthy food choices. This is primarily due to user-generated content that favors unhealthy options and the use of datasets that do not comply with health metrics like the Food Standards Agency (FSA) score and World Health Organization (WHO) guidelines.

The goal of this project is to develop a ranking food recommender system that suggests top recipe recommendations. A subsequent healthiness filter will rank these recommendations based on the healthiness of the recipes, using the FSA score. The recommendation generation will be based on the well-known Two-Tower architecture, a popular deep learning approach for building recommender systems.


The proposed model will be evaluated against a baseline: a traditional Singular Value Decomposition (SVD) collaborative filtering model. User recipe ratings will be used to assess the effectiveness of both models.

The future application of this project will involve conducting a user study to provide personalized recommendations using the Two-Tower deep learning model to predict personalized recipes according to each user preference and features. A healthiness filter algorithm will select the healthiest recipes to present to users. The key evaluative question will be:

To what extent does a highly personalized recommender system nudge users toward healthier food choices?

This notebook is organized into the following sections:

- **Dataset:** Overview of the dataset used to train the model
- **Model Architecture:** Details of the Two-Tower deep neural network
- **SVD Baseline:** Description of the baseline collaborative filtering model
- **Evaluation:** Results and evaluations of the developed models

## Dataset

The dataset that we will use in this project is collected during the course line of my PhD project and the user experiments that we condactated as mean to evaluate our propesed recommender systems and digital nudges. The dataset collected from several experiments that can found :

- [ [1] Nudging Towards Health? Examining the Merits of Nutrition
Labels and Personalization in a Recipe Recommender System](https://dl.acm.org/doi/pdf/10.1145/3503252.3531312) 
- [ [2] Boosting Health? Examining the Role of Nutrition
Labels and Preference Elicitation Methods in Food
Recommendation](https://edepot.wur.nl/579903) 
- [ [3] The Interplay between Food Knowledge, Nudges, and
Preference Elicitation Methods Determines the
Evaluation of a Recipe Recommender System](https://ceur-ws.org/Vol-3534/paper1.pdf) 
- [ [4] Nudging towards sustainable recipes: Nudging towards sustainable recipes, Internal project]()


### Data Preparation and EDA

- The dataset is from different projects and the main idea is to construct training data from the previously collected data: user features dataset and a dataframe that describes recipe features.

In [3]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### User Features
Construct a user training dataset that describes each user through various features, derived by combining different dataframes from the above projects.

In [51]:
##  user data from Nudging towards healthy food choice [1]
nudge_pers_df = pd.read_csv('./Dataset/Nudging_personalization.csv')

## user data from Boosting healthy [2]
boosting_heal_df = pd.read_csv('./Dataset/PE_boost_nudge_study.csv')

## user data from Interplay between food knowledge and preference elicitation [3]
preference_user_df = pd.read_csv('./Dataset/PE_elicitation.csv')

## sustainability data
sustainability_user_id = pd.read_csv('./Dataset/Sustainability_data.csv')

- Select useful features from each DF

In [52]:
## DF [1]
nudge_pers_df.info()
nudge_pers_df = nudge_pers_df[['person','age','country','education','diet_restriction','diet_goal', 
                               'cooking_exp','eating_habits','gender','recipe_name']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   person               300 non-null    int64 
 1   age                  300 non-null    object
 2   country              300 non-null    object
 3   education            300 non-null    object
 4   diet_restriction     300 non-null    object
 5   diet_goal            300 non-null    object
 6   cooking_exp          300 non-null    object
 7   eating_habits        300 non-null    object
 8   gender               300 non-null    object
 9   recipe_name          300 non-null    object
 10  healthiness          300 non-null    object
 11  Nutri_score          300 non-null    object
 12  fsa_score            300 non-null    int64 
 13  recommend_recipe     300 non-null    object
 14  become_favorite      300 non-null    object
 15  enjoy_eating         300 non-null    object
 16  many_to_

In [53]:
## DF [2]
boosting_heal_df.info()
boosting_heal_df = boosting_heal_df[['person','age','country','education','gender','recipe_name']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               243 non-null    int64  
 1   age                  243 non-null    object 
 2   country              243 non-null    object 
 3   education            243 non-null    object 
 4   gender               243 non-null    object 
 5   FK_1                 243 non-null    int64  
 6   FK_2                 243 non-null    int64  
 7   FK_3                 243 non-null    int64  
 8   FK_4                 243 non-null    int64  
 9   FK_5                 243 non-null    int64  
 10  category             243 non-null    object 
 11  recipe_id            243 non-null    int64  
 12  recipe_name          243 non-null    object 
 13  Nutri_score          243 non-null    object 
 14  fsa_score            243 non-null    int64  
 15  healthiness          243 non-null    obj

In [54]:
## DF [3] 
preference_user_df.info()
preference_user_df = preference_user_df[['person','age','country','education','gender','recipe_name','Height',
                                         'Weight','RecipeWebUsage','HomeCook','CookingExp',
                                         'EatingGoals','Depression','PhysicalActivity','SleepHours',
                                         'CookingTime']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               360 non-null    int64  
 1   age                  360 non-null    object 
 2   country              360 non-null    object 
 3   education            360 non-null    object 
 4   gender               360 non-null    object 
 5   FK_9                 360 non-null    int64  
 6   FK_10                360 non-null    int64  
 7   FK_11                360 non-null    int64  
 8   FK_12                360 non-null    int64  
 9   Height               180 non-null    float64
 10  Weight               180 non-null    float64
 11  RecipeWebUsage       180 non-null    object 
 12  HomeCook             180 non-null    object 
 13  CookingExp           180 non-null    object 
 14  EatingGoals          180 non-null    object 
 15  Depression           180 non-null    obj

In [55]:
## DF [4] 
sustainability_user_id.info()
sustainability_user_id = sustainability_user_id[['person','age','country','education','gender',
                                 'recipe_name','diet_restriction','diet_goal','cooking_exp','eating_habits']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               200 non-null    int64  
 1   age                  200 non-null    object 
 2   country              200 non-null    object 
 3   education            200 non-null    object 
 4   diet_restriction     200 non-null    object 
 5   diet_goal            200 non-null    object 
 6   cooking_exp          200 non-null    object 
 7   eating_habits        200 non-null    object 
 8   gender               200 non-null    object 
 9   category             200 non-null    object 
 10  recipe_name          200 non-null    object 
 11  healthiness          200 non-null    object 
 12  Nutri_score          200 non-null    object 
 13  fsa_score            200 non-null    int64  
 14  SValue_Nor           200 non-null    float64
 15  Slabel               200 non-null    obj

In [56]:
## User Dataset concatenation based on user id (person)

# change the user_id to avoid intersection of different person
boosting_heal_df[['person']] = boosting_heal_df[['person']] + nudge_pers_df[['person']].max() + 1000
preference_user_df[['person']] = preference_user_df[['person']] + boosting_heal_df[['person']].max() + 1000
sustainability_user_id[['person']] = sustainability_user_id[['person']] + preference_user_df[['person']].max() + 1000

In [57]:
print(nudge_pers_df.person.min(), nudge_pers_df.person.max())
print(boosting_heal_df.person.min(), boosting_heal_df.person.max())
print(sustainability_user_id.person.min(), sustainability_user_id.person.max())

4 2132
3148 8204
14275 16385


In [58]:
### concatenate all user data to construct user DataFrame
user_features_DF = pd.concat([nudge_pers_df, boosting_heal_df, sustainability_user_id], axis=0, ignore_index=True)
user_features_DF.to_csv('./Dataset/user_features_df.csv', index=False)

In [61]:
## Fill NAN features
user_features_DF.isna().sum()

person                0
age                   0
country               0
education             0
diet_restriction    243
diet_goal           243
cooking_exp         243
eating_habits       243
gender                0
recipe_name           0
dtype: int64

In [65]:
# features with nan values
user_features_DF[['diet_restriction','diet_goal','cooking_exp','eating_habits']] 

,diet_restriction,diet_goal,cooking_exp,eating_habits
0,No dietary restrictions,No goals,Low,Unhealthy
1,No dietary restrictions,"Eat less salt,Eat less sugar,Eat more fruit,Ea...",Medium,Neither_healthy_no_unhealthy
2,No dietary restrictions,Lose weight,Low,Neither_healthy_no_unhealthy
3,No dietary restrictions,No goals,High,Neither_healthy_no_unhealthy
4,No dietary restrictions,Eat more fruit,High,Healthy
...,...,...,...,...
738,['Lactose intolerance'],['Lose weight'],Medium,Very_unhealthy
739,['No dietary restrictions'],"['Eat less salt', ' Gain weight']",Medium,Unhealthy
740,"['Lactose intolerance', ' Vegetarian']","['No goals', ' Eat more fruit', ' Eat more pro...",High,Healthy
741,"['Gluten free', ' Lactose intolerance']",['No goals'],Low,Very_unhealthy


In [67]:
## Replace Na values
## assuming that all users will have neutral value of features with NA
user_features_DF.fillna(
    {
    'diet_restriction':'No dietary restrictions',
    'diet_goal'       :'No goals',
    'cooking_exp'     : 'Medium',
    'eating_habits'    :'Neither_healthy_no_unhealthy', 
    }
, inplace=True)

In [70]:
user_features_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743 entries, 0 to 742
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   person            743 non-null    int64 
 1   age               743 non-null    object
 2   country           743 non-null    object
 3   education         743 non-null    object
 4   diet_restriction  743 non-null    object
 5   diet_goal         743 non-null    object
 6   cooking_exp       743 non-null    object
 7   eating_habits     743 non-null    object
 8   gender            743 non-null    object
 9   recipe_name       743 non-null    object
dtypes: int64(1), object(9)
memory usage: 58.2+ KB


- The target feature from the user data is the recipe chosen by the user, represented by the recipe name

In [78]:
# extract unique selected recipes by the user
target_recipe = pd.DataFrame(user_features_DF['recipe_name'].unique(), columns=['recipes'])
target_recipe

,recipes
0,Ray's Chicken
1,Sarge's EZ Pulled Pork BBQ
2,Bow Ties with Sausage' Tomatoes and Cream
3,Scott Hibb's Amazing Whisky Grilled Baby Back ...
4,Pizza Without the Red Sauce
...,...
184,Chicken In Basil Cream
185,Rosemary Lemon Grilled Chicken
186,Beef Bulgogi
187,Honey Garlic Ribs


#### Recipe Features
Construct a training dataset that describes each recipe using various features. The original dataset, primarily sourced from Allrecipes.com, serves as the basis for our research.

In [118]:
## read original recipes
source_recipes = pd.read_csv('./Dataset/recipe_source.csv')
source_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   URL                 942 non-null    object 
 1   Name                942 non-null    object 
 2   fiber_g             942 non-null    float64
 3   sodium_g            942 non-null    float64
 4   carbohydrates_g     942 non-null    float64
 5   fat_g               942 non-null    float64
 6   protein_g           942 non-null    float64
 7   sugar_g             942 non-null    float64
 8   saturate_g          942 non-null    float64
 9   size_g              942 non-null    float64
 10  Servings            942 non-null    int64  
 11  calories_kCal       942 non-null    int64  
 12  category            942 non-null    object 
 13  image_link          942 non-null    object 
 14  fat_100g            942 non-null    float64
 15  fiber_100g          942 non-null    float64
 16  sugar_10

In [119]:
## extract features for selected recipes by users
recip_features_df  = pd.merge(source_recipes, target_recipe, right_on = 'recipes', left_on='Name')

# check if all selected recipes by users are in the orignal recipe dataset
target_recipe.recipes.isin(source_recipes.Name).any()


True

In [120]:
# drop useless columns
source_recipes.drop(columns = ['URL','image_link'], inplace=True)

In [122]:
## final recipe dataset
source_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                942 non-null    object 
 1   fiber_g             942 non-null    float64
 2   sodium_g            942 non-null    float64
 3   carbohydrates_g     942 non-null    float64
 4   fat_g               942 non-null    float64
 5   protein_g           942 non-null    float64
 6   sugar_g             942 non-null    float64
 7   saturate_g          942 non-null    float64
 8   size_g              942 non-null    float64
 9   Servings            942 non-null    int64  
 10  calories_kCal       942 non-null    int64  
 11  category            942 non-null    object 
 12  fat_100g            942 non-null    float64
 13  fiber_100g          942 non-null    float64
 14  sugar_100g          942 non-null    float64
 15  saturated_100g      942 non-null    float64
 16  protien_

## Two Tower Model
<p align="center">
  <img src="final_project.png" alt="Two-Tower Model" width="450"/>
</p>